# AirSplatMap Benchmark Analysis

Interactive analysis and visualization of benchmark results for:
- **Pose Estimation**: ATE, RPE, FPS comparison
- **Depth Estimation**: AbsRel, RMSE, accuracy metrics
- **Gaussian Splatting**: PSNR, SSIM, training efficiency

## Quick Start
1. Run the setup cells
2. If no results found, run: `python benchmarks/run_all.py --quick`
3. Explore the visualizations below

In [ ]:
import sys
from pathlib import Path
import json
import numpy as np

# Add project root
PROJECT_ROOT = Path.cwd().parent
sys.path.insert(0, str(PROJECT_ROOT))

# Check for plotting libraries
try:
    import matplotlib.pyplot as plt
    import matplotlib.cm as cm
    %matplotlib inline
    HAS_MPL = True
except ImportError:
    print("Install matplotlib: pip install matplotlib")
    HAS_MPL = False

try:
    import pandas as pd
    HAS_PANDAS = True
except ImportError:
    print("Install pandas: pip install pandas")
    HAS_PANDAS = False

print(f"Project root: {PROJECT_ROOT}")
print(f"Matplotlib: {HAS_MPL}, Pandas: {HAS_PANDAS}")

## Load Benchmark Results

In [ ]:
RESULTS_DIR = PROJECT_ROOT / "benchmarks" / "results"

def load_results(name):
    path = RESULTS_DIR / f"{name}_benchmark.json"
    if path.exists():
        with open(path) as f:
            return json.load(f)
    return []

pose_results = load_results("pose")
depth_results = load_results("depth")
gs_results = load_results("gs")

print(f"Pose results: {len(pose_results)} entries")
print(f"Depth results: {len(depth_results)} entries")
print(f"GS results: {len(gs_results)} entries")

if not any([pose_results, depth_results, gs_results]):
    print("\n⚠️  No benchmark results found!")
    print("Run: python benchmarks/run_all.py --quick")

## Pose Estimation Analysis

In [ ]:
if pose_results and HAS_PANDAS:
    df_pose = pd.DataFrame(pose_results)
    display(df_pose[['method', 'dataset', 'ate_rmse', 'rpe_trans_rmse', 'fps', 'avg_inliers']].round(4))
elif pose_results:
    print("Pose Results:")
    for r in pose_results:
        print(f"  {r['method']}: ATE={r['ate_rmse']:.4f}m, FPS={r['fps']:.1f}")
else:
    print("No pose results available")

In [ ]:
if pose_results and HAS_MPL:
    methods = [r['method'] for r in pose_results]
    ate_values = [r['ate_rmse'] for r in pose_results]
    fps_values = [r['fps'] for r in pose_results]
    
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    # ATE bar chart
    colors = plt.cm.viridis(np.linspace(0, 0.8, len(methods)))
    axes[0].bar(methods, ate_values, color=colors)
    axes[0].set_ylabel('ATE RMSE (m)')
    axes[0].set_title('Trajectory Error (lower is better)')
    axes[0].tick_params(axis='x', rotation=45)
    
    # FPS bar chart
    axes[1].bar(methods, fps_values, color=colors)
    axes[1].set_ylabel('FPS')
    axes[1].set_title('Processing Speed (higher is better)')
    axes[1].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()

## Depth Estimation Analysis

In [ ]:
if depth_results and HAS_PANDAS:
    df_depth = pd.DataFrame(depth_results)
    display(df_depth[['method', 'dataset', 'abs_rel', 'rmse', 'delta1', 'fps']].round(4))
elif depth_results:
    print("Depth Results:")
    for r in depth_results:
        print(f"  {r['method']}: AbsRel={r['abs_rel']:.4f}, δ<1.25={r['delta1']:.3f}")
else:
    print("No depth results available")

In [ ]:
if depth_results and HAS_MPL:
    methods = [r['method'] for r in depth_results]
    absrel = [r['abs_rel'] for r in depth_results]
    delta1 = [r['delta1'] for r in depth_results]
    
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    colors = plt.cm.plasma(np.linspace(0, 0.8, len(methods)))
    
    axes[0].bar(methods, absrel, color=colors)
    axes[0].set_ylabel('Abs Rel')
    axes[0].set_title('Relative Error (lower is better)')
    axes[0].tick_params(axis='x', rotation=45)
    
    axes[1].bar(methods, delta1, color=colors)
    axes[1].set_ylabel('δ < 1.25')
    axes[1].set_title('Accuracy (higher is better)')
    axes[1].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()

## Gaussian Splatting Analysis

In [ ]:
if gs_results and HAS_PANDAS:
    df_gs = pd.DataFrame(gs_results)
    cols = ['engine', 'dataset', 'psnr', 'ssim', 'final_gaussians', 'total_time', 'fps']
    available_cols = [c for c in cols if c in df_gs.columns]
    display(df_gs[available_cols].round(3))
elif gs_results:
    print("GS Results:")
    for r in gs_results:
        print(f"  {r['engine']}: PSNR={r.get('psnr', 0):.2f}dB, Gaussians={r.get('final_gaussians', 0):,}")
else:
    print("No GS results available")

In [ ]:
if gs_results and HAS_MPL:
    engines = [r['engine'] for r in gs_results]
    psnr = [r.get('psnr', 0) for r in gs_results]
    gaussians = [r.get('final_gaussians', 0) for r in gs_results]
    
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    colors = plt.cm.cool(np.linspace(0, 0.8, len(engines)))
    
    axes[0].bar(engines, psnr, color=colors)
    axes[0].set_ylabel('PSNR (dB)')
    axes[0].set_title('Reconstruction Quality (higher is better)')
    
    axes[1].bar(engines, gaussians, color=colors)
    axes[1].set_ylabel('# Gaussians')
    axes[1].set_title('Model Size')
    axes[1].ticklabel_format(style='scientific', axis='y', scilimits=(0,0))
    
    plt.tight_layout()
    plt.show()

## Run New Benchmarks

Execute benchmark suite directly from notebook:

In [ ]:
# Uncomment to run benchmarks:

# !python {PROJECT_ROOT}/benchmarks/run_all.py --quick --pose
# !python {PROJECT_ROOT}/benchmarks/run_all.py --quick --depth  
# !python {PROJECT_ROOT}/benchmarks/run_all.py --quick --gs

## Export Summary

In [ ]:
# Create summary
summary = {
    'pose': {
        'best_accuracy': min(pose_results, key=lambda x: x['ate_rmse'])['method'] if pose_results else 'N/A',
        'fastest': max(pose_results, key=lambda x: x['fps'])['method'] if pose_results else 'N/A',
    },
    'depth': {
        'best_accuracy': min(depth_results, key=lambda x: x['abs_rel'])['method'] if depth_results else 'N/A',
        'fastest': max(depth_results, key=lambda x: x['fps'])['method'] if depth_results else 'N/A',
    },
    'gs': {
        'best_quality': max(gs_results, key=lambda x: x.get('psnr', 0))['engine'] if gs_results else 'N/A',
        'fastest': max(gs_results, key=lambda x: x.get('fps', 0))['engine'] if gs_results else 'N/A',
    }
}

print("=" * 50)
print("BENCHMARK SUMMARY")
print("=" * 50)
print(f"\n📍 Pose Estimation:")
print(f"   Best accuracy: {summary['pose']['best_accuracy']}")
print(f"   Fastest: {summary['pose']['fastest']}")
print(f"\n🎯 Depth Estimation:")
print(f"   Best accuracy: {summary['depth']['best_accuracy']}")
print(f"   Fastest: {summary['depth']['fastest']}")
print(f"\n✨ Gaussian Splatting:")
print(f"   Best quality: {summary['gs']['best_quality']}")
print(f"   Fastest: {summary['gs']['fastest']}")